MÉTRICAS DE COMPARAÇÃO E TESTES DEHIPÓTESES

In [19]:
from google.colab import files
uploaded = files.upload()


Saving Metricas_Similaridade_Resultado.xlsx to Metricas_Similaridade_Resultado.xlsx


In [2]:
pip install levenshtein


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.3 MB/s eta 0:00:00


In [13]:

# Carregar arquivo CSV do Google Drive
file_path = '/content/BASE_GPTRAG5.csv'

In [5]:
pip install pandas nltk python-Levenshtein rouge-score bert-score transformers


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.7 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=f2e8b6e6ff9dcde511be2772ad7de7e7a539e5106565a053e781c022dd6066a7
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [18]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from bert_score import score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Carregar o arquivo Excel
file_path = 'BASE_GPTRAG5.xlsx'
df = pd.read_excel(file_path)

# Listas de variáveis
columns_ref = [
    'Base_Normativa_REF', 'Revogada_REF', 'Dispoe_REF', 'Cargo_REF',
    'Instrucoes_preenchimento_REF', 'Prazo_de_envio_REF', 'Obrigacao_envio_REF'
]
columns_rag = [
    'Base_Normativa_MOD1', 'Revogada_MOD1', 'Dispoe_MOD1', 'Cargo_MOD1',
    'Instrucoes_preenchimento_MOD1', 'Prazo_de_envio_MOD1', 'Obrigacao_envio_MOD1'
]
columns_gpt = [
    'Base_Normativa_MOD2', 'Revogada_MOD2', 'Dispoe_MOD2', 'Cargo_MOD2',
    'Instrucoes_preenchimento_MOD2', 'Prazo_de_envio_MOD2', 'Obrigacao_envio_MOD2'
]

# Garantir que as listas sejam do mesmo tamanho
assert len(columns_ref) == len(columns_rag) == len(columns_gpt), "As listas REF, RAG e GPT devem ter o mesmo número de elementos."

# Configurar o modelo BLEURT
bleurt_model_name = "Elron/bleurt-base-512"
bleurt_tokenizer = AutoTokenizer.from_pretrained(bleurt_model_name)
bleurt_model = AutoModelForSequenceClassification.from_pretrained(bleurt_model_name)

def bert_scores(reference, candidate):
    if pd.notnull(reference) and pd.notnull(candidate):
        P, R, F1 = score([candidate], [reference], lang="pt")
        return P.mean().item(), R.mean().item(), F1.mean().item()
    return None, None, None

def cosine_similarity_score(text1, text2):
    if pd.notnull(text1) and pd.notnull(text2):
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform([text1, text2])
        cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])
        return cosine_sim[0][0]
    return None

# Comparar REF, RAG e GPT
for ref_col, rag_col, gpt_col in zip(columns_ref, columns_rag, columns_gpt):
    # REF x RAG
    bert_rag = df.apply(lambda row: bert_scores(row[ref_col], row[rag_col]), axis=1)
    df[f'BERT_Precision_{ref_col}_{rag_col}'] = bert_rag.apply(lambda x: x[0] if x else None)
    df[f'BERT_Recall_{ref_col}_{rag_col}'] = bert_rag.apply(lambda x: x[1] if x else None)
    df[f'BERT_F1_{ref_col}_{rag_col}'] = bert_rag.apply(lambda x: x[2] if x else None)
    df[f'Cosine_{ref_col}_{rag_col}'] = df.apply(lambda row: cosine_similarity_score(row[ref_col], row[rag_col]), axis=1)

    # REF x GPT
    bert_gpt = df.apply(lambda row: bert_scores(row[ref_col], row[gpt_col]), axis=1)
    df[f'BERT_Precision_{ref_col}_{gpt_col}'] = bert_gpt.apply(lambda x: x[0] if x else None)
    df[f'BERT_Recall_{ref_col}_{gpt_col}'] = bert_gpt.apply(lambda x: x[1] if x else None)
    df[f'BERT_F1_{ref_col}_{gpt_col}'] = bert_gpt.apply(lambda x: x[2] if x else None)
    df[f'Cosine_{ref_col}_{gpt_col}'] = df.apply(lambda row: cosine_similarity_score(row[ref_col], row[gpt_col]), axis=1)

    # RAG x GPT
    bert_rag_gpt = df.apply(lambda row: bert_scores(row[rag_col], row[gpt_col]), axis=1)
    df[f'BERT_Precision_{rag_col}_{gpt_col}'] = bert_rag_gpt.apply(lambda x: x[0] if x else None)
    df[f'BERT_Recall_{rag_col}_{gpt_col}'] = bert_rag_gpt.apply(lambda x: x[1] if x else None)
    df[f'BERT_F1_{rag_col}_{gpt_col}'] = bert_rag_gpt.apply(lambda x: x[2] if x else None)
    df[f'Cosine_{rag_col}_{gpt_col}'] = df.apply(lambda row: cosine_similarity_score(row[rag_col], row[gpt_col]), axis=1)

# Salvar resultados em um novo arquivo
output_file = 'Metricas_Similaridade_Resultado.xlsx'
df.to_excel(output_file, index=False)

print(f"Arquivo com métricas salvo em: {output_file}")

Arquivo com métricas salvo em: Metricas_Similaridade_Resultados.xlsx


In [20]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import ttest_rel, sem, t

file_path = 'Metricas_Similaridade_Resultado.xlsx'
# Ler o arquivo Excel
data = pd.read_excel(file_path)


# Lista de pares de colunas a serem comparadas
pairs = [
    ("BERT_Precision_REF_MOD1", "BERT_Precision_REF_MOD2"),
    ("BERT_Recall_REF_MOD1", "BERT_Recall_REF_MOD2"),
    ("BERT_F1_REF_MOD1", "BERT_F1_REF_MOD2"),
    ("Cosine_REF_MOD1", "Cosine_REF_MOD2"),
]

# Criar uma lista para salvar os resultados dos testes
test_results = []

# Lista para armazenar as figuras
figures = []

# Criar lista para métricas estatísticas
stats_summary = []

def calculate_confidence_interval(data, confidence=0.95):
    n = len(data)
    m = np.mean(data)
    std_err = sem(data, nan_policy='omit')
    h = std_err * t.ppf((1 + confidence) / 2, n - 1)
    return m - h, m + h

for pair in pairs:
    col1, col2 = pair

    # Criar boxplot
    plt.figure()
    data[[col1, col2]].boxplot(
        boxprops=dict(linewidth=1.5, color="black"),
        whiskerprops=dict(linewidth=1.5, color="black"),
        capprops=dict(linewidth=1.5, color="black"),
        medianprops=dict(linewidth=1.5, color="red")
    )
    plt.title(f"Comparativo: {col1} x {col2}", fontsize=12)
    plt.ylabel("Valor", fontsize=10)
    plt.xticks(fontsize=9)
    plt.yticks(fontsize=9)
    plt.grid(False)
    plt.savefig(f"{col1}_vs_{col2}_boxplot.png", dpi=300, bbox_inches='tight')
    figures.append(plt.gcf())
    plt.close()

    # Teste pareado
    t_stat, p_value = ttest_rel(data[col1], data[col2], nan_policy='omit')
    test_results.append({
        "Pair": f"{col1} x {col2}",
        "T-Statistic": t_stat,
        "P-Value": p_value
    })

    # Estatísticas descritivas
    for col in [col1, col2]:
        mean = data[col].mean()
        median = data[col].median()
        std_dev = data[col].std()
        ci_lower, ci_upper = calculate_confidence_interval(data[col])
        stats_summary.append({
            "Column": col,
            "Mean": mean,
            "Median": median,
            "Standard Deviation": std_dev,
            "CI Lower": ci_lower,
            "CI Upper": ci_upper
        })

# Converter resultados para DataFrame
test_results_df = pd.DataFrame(test_results)
stats_summary_df = pd.DataFrame(stats_summary)

# Criar figura com todos os boxplots
plt.figure(figsize=(12, 8))
for i, pair in enumerate(pairs, 1):
    col1, col2 = pair
    plt.subplot(2, 2, i)
    data[[col1, col2]].boxplot(
        boxprops=dict(linewidth=1.5, color="black"),
        whiskerprops=dict(linewidth=1.5, color="black"),
        capprops=dict(linewidth=1.5, color="black"),
        medianprops=dict(linewidth=1.5, color="red")
    )
    plt.title(f"{col1} x {col2}", fontsize=12)
    plt.ylabel("Valor", fontsize=10)
    plt.xticks(fontsize=9)
    plt.yticks(fontsize=9)
    plt.grid(False)
plt.tight_layout()
plt.savefig("all_boxplots.png", dpi=300, bbox_inches='tight')
plt.close()

# Salvar resultados dos testes e estatísticas descritivas em uma planilha


output_file = 'test.xlsx'
with pd.ExcelWriter(output_file) as writer:
    test_results_df.to_excel(writer, sheet_name="Test Results", index=False)
    stats_summary_df.to_excel(writer, sheet_name="Statistics Summary", index=False)

print(f"Resultados salvos em: {output_file}")



Resultados salvos em: test.xlsx
